In [1]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion_model")
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds

2023-01-24 13:53:38.700851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 13:53:39.408506: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 13:53:39.408568: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-24 13:53:39.408574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [15]:
importlib.reload(w2vU)
importlib.reload(lds)
tess_dataset = lds.AudioEmotionTessDataset("/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device="cpu")
#trainSet, testSet = ADU.train_val_dataset(tess_dataset)
processor, sr = w2vU.init_w2v2(num_labels=len(tess_dataset.label_list), label_list=tess_dataset.label_list)

['OAF', 'dime', 'disgust.flac'] OAF_dime_disgust.flac
['OAF', 'haze', 'disgust.flac'] OAF_haze_disgust.flac
['OAF', 'thumb', 'disgust.flac'] OAF_thumb_disgust.flac
['OAF', 'death', 'disgust.flac'] OAF_death_disgust.flac
['OAF', 'rot', 'disgust.flac'] OAF_rot_disgust.flac
['OAF', 'chat', 'disgust.flac'] OAF_chat_disgust.flac
['OAF', 'far', 'disgust.flac'] OAF_far_disgust.flac
['OAF', 'date', 'disgust.flac'] OAF_date_disgust.flac
['OAF', 'red', 'disgust.flac'] OAF_red_disgust.flac
['OAF', 'hush', 'disgust.flac'] OAF_hush_disgust.flac
['OAF', 'goal', 'disgust.flac'] OAF_goal_disgust.flac
['OAF', 'pain', 'disgust.flac'] OAF_pain_disgust.flac
['OAF', 'sure', 'disgust.flac'] OAF_sure_disgust.flac
['OAF', 'bar', 'disgust.flac'] OAF_bar_disgust.flac
['OAF', 'hire', 'disgust.flac'] OAF_hire_disgust.flac
['OAF', 'search', 'disgust.flac'] OAF_search_disgust.flac
['OAF', 'dip', 'disgust.flac'] OAF_dip_disgust.flac
['OAF', 'five', 'disgust.flac'] OAF_five_disgust.flac
['OAF', 'calm', 'disgust.flac'

In [3]:
import datasets

importlib.reload(lds)

loaded = tess_dataset[500][0].cpu()
samplerate = tess_dataset.samplerate

newSet = lds.AudioEmotionTessWav2VecDataset(tess_dataset, processor= processor, sampling_rate=sr)
#res = torchaudio.transforms.Resample(orig_freq=samplerate, new_freq=sr)(loaded)
#output = processor(res)
#processor.pad()

In [4]:
#newSet.getEmotionFromId(np.where(newSet[1][1] == 1))

In [5]:
import network_models.w2v_emotion_model.custom_model as cm
import network_models.soundstream_lstm.model_trainer as ct

model = cm.W2V_EmotionClassifierSevenEmos().to(device=device)

In [7]:
importlib.reload(ADU)
trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)

In [14]:
newSet.dataSet.label_list

array(['angry', 'disgust', 'fear', 'happy', 'neutral', 'ps', 'sad'],
      dtype=object)

In [11]:
importlib.reload(ct)
trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=16, save_model_every=500, num_epochs=2001, model_path="content/classifier/W2VClassifier/Nr1")

In [12]:
trainer.train()

2482
Epoch 1
-------------------------------
loss: 1.943748  [    0/ 2482]
loss: 1.959775  [ 1600/ 2482]
Test Error: 
 Accuracy: 17.8%, Avg loss: 1.944720 

Epoch 2
-------------------------------
loss: 1.927813  [    0/ 2482]
loss: 1.946969  [ 1600/ 2482]
Test Error: 
 Accuracy: 18.1%, Avg loss: 1.943899 

Epoch 3
-------------------------------
loss: 1.927309  [    0/ 2482]
loss: 1.962918  [ 1600/ 2482]
Test Error: 
 Accuracy: 19.6%, Avg loss: 1.942961 

Epoch 4
-------------------------------
loss: 1.954435  [    0/ 2482]
loss: 1.943867  [ 1600/ 2482]
Test Error: 
 Accuracy: 20.7%, Avg loss: 1.942083 

Epoch 5
-------------------------------
loss: 1.920491  [    0/ 2482]
loss: 1.932721  [ 1600/ 2482]
Test Error: 
 Accuracy: 21.7%, Avg loss: 1.941203 

Epoch 6
-------------------------------
loss: 1.933069  [    0/ 2482]
loss: 1.937573  [ 1600/ 2482]
Test Error: 
 Accuracy: 22.5%, Avg loss: 1.940193 

Epoch 7
-------------------------------
loss: 1.914533  [    0/ 2482]
loss: 1.89881

In [ ]:
from IPython.lib.display import Audio
Audio(loaded, rate=samplerate)